In [14]:
from langchain.agents import AgentExecutor, BaseMultiActionAgent, Tool
from langchain_community.utilities import SerpAPIWrapper
import os

os.environ['SERPAPI_API_KEY'] = '

In [15]:
def random_word(query: str) -> str:
    print("\nNow I'm doing this!")
    return "foo"

In [16]:
search = SerpAPIWrapper()

tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="random_word",
        func=random_word,
        description="call this to get a random word.",
    ),
]

In [17]:
from typing import Any, List, Tuple, Union
from langchain.schema import AgentAction, AgentFinish
from langchain_core.callbacks import Callbacks

class MyAgent(BaseMultiActionAgent):
    """Fake Custom Agent"""

    @property
    def input_keys(self):
        return ["input"]
    
    def plan(
            self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any 
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="random_word", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(return_values={"output": "bar"}, log="")
    
    async def aplan(
            self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
            ) -> List[AgentAction] | AgentFinish:
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="random_word", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(return_values={"output": "bar"}, log="")

In [18]:
agent = MyAgent()
agent_executor = AgentExecutor.from_agent_and_tools(agent, tools, verbose=True)

In [19]:
agent_executor.run(input="What is the weather in New York?")



> Entering new AgentExecutor chain...
{'type': 'weather_result', 'temperature': '39', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '73%', 'wind': '5 mph', 'location': 'New York, NY', 'date': 'Tuesday 2:00 PM', 'weather': 'Cloudy'}
Now I'm doing this!
foo

> Finished chain.


'bar'